# ECE 498 DS Mini Project 2

## Akhilesh Somani - somani4,
## Gowtham Kuntumalla - gowtham4,
## Manan Mehta - mananm2

# Task 1: Data Cleaning and Visual Inspection

## 0. Getting Started With the Data

In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [11]:
HE0_df=pd.read_csv("RelativeAbundance_HE0.csv")

In [12]:
HE0_df.head(10)

,Microbe Name,HE0Sample_0,HE0Sample_1,HE0Sample_2,HE0Sample_3,HE0Sample_4,HE0Sample_5,HE0Sample_6,HE0Sample_7,HE0Sample_8,...,HE0Sample_754,HE0Sample_755,HE0Sample_756,HE0Sample_757,HE0Sample_758,HE0Sample_759,HE0Sample_760,HE0Sample_761,HE0Sample_762,HE0Sample_763
0,Acidobacteria_Acidobacteria_Gp1_Telmatobacter_...,0.004239,0.004345,0.004423,0.004463,0.005372,0.004136,0.004457,0.004049,0.004638,...,0.004464,0.004636,0.004046,0.005518,0.005073,0.003527,0.005413,0.003140,0.002998,0.004829
1,Acidobacteria_Acidobacteria_Gp3_Gp3_Gp3,0.013269,0.012312,0.011611,0.013033,0.013689,0.012752,0.009772,0.012148,0.013743,...,0.010603,0.013565,0.012139,0.012875,0.013294,0.014107,0.012694,0.013082,0.008244,0.010401
2,Actinobacteria_Actinobacteria_Acidimicrobiales...,0.003502,0.005251,0.006082,0.003749,0.004332,0.004136,0.004972,0.004930,0.004638,...,0.003906,0.002919,0.004782,0.006070,0.003498,0.004408,0.004853,0.004012,0.004310,0.005758
3,Actinobacteria_Actinobacteria_Actinomycetales_...,0.003870,0.005794,0.005345,0.004106,0.002946,0.003963,0.005315,0.003697,0.005326,...,0.005394,0.004979,0.004598,0.005518,0.005597,0.004761,0.004293,0.005233,0.004497,0.005386
4,Actinobacteria_Actinobacteria_Actinomycetales_...,0.004976,0.003802,0.004239,0.004285,0.005198,0.004136,0.003257,0.002993,0.003264,...,0.004092,0.003777,0.004782,0.004598,0.003848,0.004585,0.003360,0.004186,0.004684,0.002043
5,Actinobacteria_Actinobacteria_Actinomycetales_...,0.012532,0.010683,0.011426,0.011427,0.014902,0.011373,0.011143,0.014789,0.011854,...,0.013393,0.011504,0.011035,0.012323,0.013119,0.010580,0.014747,0.010466,0.011055,0.012444
6,Actinobacteria_Actinobacteria_Actinomycetales_...,0.004055,0.004345,0.003502,0.003571,0.003639,0.003963,0.003429,0.004577,0.004295,...,0.005208,0.003949,0.004598,0.005518,0.005422,0.003350,0.005040,0.004535,0.003185,0.004643
7,Actinobacteria_Actinobacteria_Actinomycetales_...,0.004055,0.006156,0.004792,0.004999,0.004159,0.005859,0.004286,0.004754,0.003264,...,0.003906,0.003434,0.005334,0.003495,0.005422,0.004408,0.004107,0.003837,0.003560,0.006315
8,Actinobacteria_Actinobacteria_Actinomycetales_...,0.004792,0.004527,0.005345,0.004642,0.003812,0.002930,0.003772,0.002641,0.006185,...,0.005394,0.004293,0.005886,0.005702,0.002799,0.006524,0.004107,0.004884,0.004310,0.006501
9,Actinobacteria_Actinobacteria_Actinomycetales_...,0.005898,0.005251,0.004239,0.004821,0.003292,0.004997,0.005829,0.006338,0.005326,...,0.002976,0.005151,0.004414,0.003862,0.004023,0.004937,0.004667,0.004535,0.005621,0.005201


### 1. In the context of statistical analysis, why do biologists need multiple samples to identify microbes with significantly altered abundance?

Biologists need muliple samples to reduce the standard deviation of the data correpsonding to each microboe. This helps them to conclude, with a greater confidence, which mircobes are present in more numbers than usual.

### 2. How many samples were analyzed?
Number of samples analyzed were: 764

In [21]:
len(HE0_df.set_index("Microbe Name").columns)

764

### 3. How many microbes were identified?
Number of different micorbes idenified were: 149

In [26]:
HE0_df.groupby("Microbe Name").count().sum()[0]

149

## 1. Bayesian Network for Quality Control

### a. Joint Probability Factorization
T = storage temp,
M = collection method,
C = contamination,
L = Lab time,
Q = quality

$$ P(joint) = P(Q,C,L,M,T) = P(Q|C,L,M,T) * P(C|M,T,L) * P(M|T,L) * P(T|L) * P(L)$$ 
$$ P(Q,C,L,M,T) = P(Q|C,L) * P(C|M,T) * P(M) * P(T) * P(L)$$

### b. Parameters Required
Number of values taken by:
C=2,
L=2,
T=2,
M=2,
Q=2

Parameters needed are: 2x2x1x1x1 = 4

### c. Constructing Probability Tables

### d. Final Probability Tables

### e. Cleaning the dataset

## 2. Data Standardization

### 1. Verify Relative Abundance

### 2. Discussion on Relative Abundance Data

## 3. Visual Inspection

# Task 2: Statistical Analysis

## 1. Kolmogorov-Smirnov (KS) Test

### a. p-value for Two-Sample KS Test

### b. Null Hypothesis of the KS Test in our Context

### c. Number of Genes with Significantly Altered Expression at Various Alpha Values

## 2. Multiple Testing

### a. P-value

### b. P-value Distribution

### c. Expected Number of Significant p-values

### d. Q-Q plot of p-values

#### i. 

#### ii. 

#### iii. 

#### iv. 

#### v. 

#### vi. 

### e. Follow-up Questions

#### i. 

#### ii. 

# Task 3: Dimensionality Reduction and Clustering

## 1. PCA

### a. Performing PCA


### b. Ordering Components

### c. Scatter Plot of Microbe Abundance

## 2. t-SNE

### a. Performing t-SNE

### b. Comparing with PCA

## 3. Clustering

### a. K-Means

### b. GMM

### c. Single and Complete Linkage Hierarchical

### d. Single vs Complete Link Hierarchical Clustering

### e. Comparing Clustering Techniques

### f. What do Clusters Represent

### g. Deciding on Number of Clusters

# Task 4: Interpreting Results

## 1. Identify Microbes with Altered Abundance Levels

### a. Determine if each HE1 Subpopulation shows altered abundance levels

### b. Identify Most Similar HE0 Clusters to Affected HE1 Clusters

### c. Identify Microbes with Significantly Altered Relative Abundance

## 2. Identify How Abundance Changes

### a. Microbes which increase and decrease

### b. Biological Relations